In [1]:
import sqlalchemy as sqlA
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session
from sqlalchemy import Column
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import Float
from sqlalchemy import ForeignKey
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import select
from sqlalchemy import func
import pprint

In [2]:
Base = declarative_base()

In [3]:
# Create Tables

class Client(Base):
  __tablename__ = "client"
  # attributes
  id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String(50), nullable=False)
  document = Column(String(11), nullable=False)
  address= Column(String(100), nullable=False)

  account = relationship(
      "Account", back_populates="client", cascade = "all, delete-orphan"
  )

  def __repr__(self):
    return f"Client(id={self.id}, name={self.name}, document={self.document}, document={self.address})"

class Account(Base):
  __tablename__ = "account"
  # attributes
  id = Column(Integer, primary_key=True, autoincrement=True)
  type_acc = Column(String, nullable=False)
  agency = Column(Integer, nullable=False)
  acc_number = Column(Integer, nullable=False)
  balance = Column(Float, nullable=False)
  # A check account belongs to a client
  id_cli = Column(Integer, ForeignKey("client.id"), nullable=False)
  
  client = relationship(
      "Client", back_populates="account"
  )

  def __repr__(self):
    return f"Account(id={self.id}, type_acc={self.type_acc}, agency={self.agency}, account={self.acc_number}, balance={self.balance}, id_cli={self.id_cli})"


In [4]:
# print the name of created tables
print(Client.__tablename__)
print(Account.__tablename__)

client
account


In [5]:
# create a connection 
engine = create_engine("sqlite://")

In [6]:
# generating tables from the classes
Base.metadata.create_all(engine)

In [7]:
# inspect the schema
inspect_engine = inspect(engine)

print(inspect_engine.get_table_names())
print(inspect_engine.default_schema_name)

['account', 'client']
main


In [8]:
# create an instance
with Session(engine) as session:
  maria = Client(
      name = "Maria da Silva",
      document = "9999999999999",
      address = "100, Fifth Avenue, Santa Barbara - CA",
      account = [Account(type_acc="C", agency=111, acc_number=123456, balance=50.55)]
  )
  jose = Client(
      name = "Jose Pereira",
      document = "8888888888888",
      address = "22, President Kennedy Street, Orlando - FL",
      account = [Account(type_acc="C", agency=111, acc_number=123456, balance=50000.55),
                 Account(type_acc="S", agency=111, acc_number=222333, balance=500.55)]
  )
  pedro = Client(
      name = "Pedro Ribeiro",
      document = "1234567890987",
      address = "1523, , Flower Street, New York - NY"
  )

In [9]:
# add instances to the session
session.add_all([maria, jose, pedro])
session.commit()


In [10]:
# search for client
stmt = select(Client).where(Client.name.in_(["Maria da Silva", "Pedro Ribeiro"]))
for item in session.scalars(stmt):
  print(item)


Client(id=1, name=Maria da Silva, document=9999999999999, document=100, Fifth Avenue, Santa Barbara - CA)
Client(id=3, name=Pedro Ribeiro, document=1234567890987, document=1523, , Flower Street, New York - NY)


In [11]:
# search for accounts
stmt = select(Account).where(Account.id_cli.in_([2]))
for item in session.scalars(stmt):
  print(item)

Account(id=2, type_acc=C, agency=111, account=123456, balance=50000.55, id_cli=2)
Account(id=3, type_acc=S, agency=111, account=222333, balance=500.55, id_cli=2)


In [12]:
# search for jose account
stmt = select(Client).where(Account.id_cli.in_([1]))
for item in session.scalars(stmt):
  print(item)

Client(id=1, name=Maria da Silva, document=9999999999999, document=100, Fifth Avenue, Santa Barbara - CA)
Client(id=2, name=Jose Pereira, document=8888888888888, document=22, President Kennedy Street, Orlando - FL)
Client(id=3, name=Pedro Ribeiro, document=1234567890987, document=1523, , Flower Street, New York - NY)


<ipython-input-12-2fa39505aa29>:3: SAWarning: SELECT statement has a cartesian product between FROM element(s) "account" and FROM element "client".  Apply join condition(s) between each element to resolve.
  for item in session.scalars(stmt):


In [13]:
# search for client order by name (desc) descendant or (asc) ascendant
stmt = select(Client).order_by(Client.name.desc())
for item in session.scalars(stmt):
  pprint.pprint(item)

Client(id=3, name=Pedro Ribeiro, document=1234567890987, document=1523, , Flower Street, New York - NY)
Client(id=1, name=Maria da Silva, document=9999999999999, document=100, Fifth Avenue, Santa Barbara - CA)
Client(id=2, name=Jose Pereira, document=8888888888888, document=22, President Kennedy Street, Orlando - FL)


In [27]:
# search for client and related account
stmt = select(Client.name, Account.acc_number).join_from(Client, Account)
for item in session.scalars(stmt):
  pprint.pprint(item)

'Maria da Silva'
'Jose Pereira'
'Jose Pereira'


In [14]:
# # execute a statement in a connection from account and related client
stmt = select(Account.acc_number, Client.name).join_from(Account, Client)
connection = engine.connect()
results = connection.execute(stmt).fetchall()
for result in results:
  pprint.pprint(result)

(123456, 'Maria da Silva')
(123456, 'Jose Pereira')
(222333, 'Jose Pereira')


In [15]:
# execute a statement in a connection from client and related account
stmt = select(Client.name, Account.acc_number).join_from(Client, Account)
connection = engine.connect()
results = connection.execute(stmt).fetchall()
for result in results:
  pprint.pprint(result)

('Maria da Silva', 123456)
('Jose Pereira', 123456)
('Jose Pereira', 222333)


In [16]:
# counting items from a result
stmt = select(func.count('*')).select_from(Client)
for result in session.scalars(stmt):
  print(result)

3


In [17]:
# closing the session
session.close()